In [2]:
from ROOT import *
import numpy as np
import math
import time

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

gROOT.ProcessLine(".x ~/lhcbStyle.C")
gStyle.SetPaintTextFormat("1.3f")



Welcome to JupyROOT 6.14/04


/cvmfs/sft.cern.ch/lcg/views/LCG_94/x86_64-centos7-gcc7-opt/lib/ROOT.py:577: UserWarning: "from ROOT import *" is not supported under IPython
  warnings.warn( '"from ROOT import *" is not supported under IPython' )


In [10]:
#OLD FILE VELO UT CONFIG

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_oldfile_20ev.root")
ch2=TChain("MCParticleNTuple/Tracks")
ch2.Add("~/MightyIT/MCtracks_oldfile_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_oldfile_nocuts_20ev.root")
#ch2=TChain("MCParticleNTuple/Tracks")
#ch2.Add("~/MightyIT/MCtracks_oldfile_nocuts_20ev.root")

zmag = 5150. #for MCtracks_oldfile WITH UT


#no smearing sigmas for oldfile
sigmax_nosm=6.3
sigmay_nosm=3.9
sigmatx_nosm=0.0022
sigmaty_nosm=0.0009

sigmax_wsm=55
sigmay_wsm=4
sigmatx_wsm=0.02
sigmaty_wsm=0.06



perc_list = [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
thr_wsm_list = [0.96, 1.08, 1.21, 1.32, 1.44, 1.61, 1.79, 1.98, 2.28, 2.65, 3.14, 3.74, 4.62, 5.57, 7.03, 8.93, 11.58, 15.11, 20.76, 40.05]
thr_nosm_list = [3.05, 3.33, 3.64, 3.98, 4.32, 4.76, 5.29, 5.94, 6.66, 7.5, 8.8, 10.19, 12.14, 15.2, 18.29, 22.06, 28.85, 40.16, 60.07, 95.15]



In [17]:
# NEW FILE VELO UT CONFIG

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")
ch2=TChain("MCParticleNTuple/Tracks")
ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")
#ch2=TChain("MCParticleNTuple/Tracks")
#ch2.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")


zmag = 5144.5 #for MCtracks_MagUp_bs2phiphi_1p5e34 WITH UT

sigmax_nosm=9.2
sigmay_nosm=4.4
sigmatx_nosm=0.0034
sigmaty_nosm=0.0012

sigmax_wsm=24
sigmay_wsm=4.5
sigmatx_wsm=0.009
sigmaty_wsm=0.02

perc_list = [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
thr_wsm_list = [1.15, 1.24, 1.34, 1.52, 1.69, 1.89, 2.07, 2.31, 2.61, 3.03, 3.55, 4.18, 4.86, 6.18, 7.72, 10.28, 14.67, 21.02, 33.97, 56.34]
thr_nosm_list = [2.42, 2.63, 2.85, 3.09, 3.42, 3.73, 4.1, 4.54, 5.05, 5.69, 6.63, 7.82, 8.94, 11.23, 13.73, 17.84, 23.09, 32.18, 44.13, 68.5]









In [4]:
# pixel smearing in mightyIT
resx_mit = 0.1/TMath.Sqrt(12)
resy_mit = 0.4/TMath.Sqrt(12)
# strips smearing outside mightyIT
resx_sf = 0.25/TMath.Sqrt(12)
resy_sf = 0.25/TMath.Cos(5.*np.pi/180.) #takes into account 5° tilting


In [5]:
#function that checks if a point (format: (x,y)) is inside the MightyIT region

def MightyITregion():
    dx = 540
    dy = 200
    
    polygon = Polygon([(-4*dx,1*dy),(-3*dx,1*dy),(-3*dx,2*dy),(-2*dx,2*dy),(-2*dx,3*dy),(-1*dx,3*dy),(-1*dx,4*dy),
                      (1*dx,4*dy),(1*dx,3*dy),(2*dx,3*dy),(2*dx,2*dy),(3*dx,2*dy),(3*dx,1*dy),(4*dx,1*dy),
                      (4*dx,-1*dy),(3*dx,-1*dy),(3*dx,-2*dy),(2*dx,-2*dy),(2*dx,-3*dy),(1*dx,-3*dy),(1*dx,-4*dy),
                      (-1*dx,-4*dy),(-1*dx,-3*dy),(-2*dx,-3*dy),(-2*dx,-2*dy),(-3*dx,-2*dy),(-3*dx,-1*dy),(-4*dx,-1*dy)])
    return polygon
    
def is_inregion(point):
    point = Point(point[0], point[1])
           
    region = MightyITregion()
    if region.contains(point):
        return True
    else:
        return False
    

In [6]:
#function which calculates the chi2, also checks if t track is in window if flag is set True

def ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, rangex, rangey, sigma2x, sigma2y, sigma2tx, sigma2ty, check_if_inwindow=False, smearing=False): #to ask: velo p?
    
    #gaussian smearing - pixel resolutions x and y according to if in mightyit region or outside
    if smearing == True:
        if (is_inregion([xt0,yt0]) == True):
            xt0 = gRandom.Gaus(xt0,resx_mit)
            yt0 = gRandom.Gaus(yt0,resy_mit)
        else:
            xt0 = gRandom.Gaus(xt0,resx_sf)
            yt0 = gRandom.Gaus(yt0,resy_sf)
            
        if (is_inregion([xt1,yt1]) == True):
            xt1 = gRandom.Gaus(xt1,resx_mit)
            yt1 = gRandom.Gaus(yt1,resy_mit)
        else:
            xt1 = gRandom.Gaus(xt1,resx_sf)
            yt1 = gRandom.Gaus(yt1,resy_sf)
            
    tyV=(yv1 - yv0)/(zv1 - zv0)
    txV=(xv1 - xv0)/(zv1 - zv0)
    tyT=(yt1 - yt0)/(zt1 - zt0)
    txT=(xt1 - xt0)/(zt1 - zt0)

    #zmag calculations
    xVzmag=xv1+(zmag-zv1)*txV
    yVzmag=yv1+(zmag-zv1)*tyV
       
    
    #chi2 calculations
    txPre=(xt0-xVzmag)/(zt0-zmag)
    tyPre=(yt0-yVzmag)/(zt0-zmag)
    
    #extrapolated point from zmag to t station, with the same slope of the t track
    xT_ex=xVzmag+(zt0-zmag)*txT
    #yT_ex=yVzmag+(zt0-zmag)*tyT
    #from velo to t track with velo slope
    yT_ex=yv1+(zt0-zv1)*tyV

    #ŧesting: complete chi2 (slopes and coordinates)
    chi2 = (txPre-txT)*(txPre-txT)/sigma2tx+(tyPre-tyT)*(tyPre-tyT)/sigma2ty + (yT_ex-yt0)*(yT_ex-yt0)/sigma2y + (xT_ex-xt0)*(xT_ex-xt0)/sigma2x 

    #check if in window
    if (check_if_inwindow == True):
        

        #if in window: return chi2, else placeholder -1
        if(abs(xT_ex-xt0)<rangex and abs(yT_ex-yt0)<rangey):
            return chi2, (xT_ex-xt0), (yT_ex-yt0), (txPre-txT), (tyPre-tyT)
        else:
            return (-1, -1, -1, -1, -1)
            
    #no need to check 
    else:
        return chi2, (xT_ex-xt0), (yT_ex-yt0), (txPre-txT), (tyPre-tyT) 

In [19]:
smearing = False

for percentage in [97]:
#for percentage in [99,98,97,96,95]:

    ##### INITIALISE VARIABLES #####
    
    timestart = time.time()
    
    nLongTracks = 0
    nVeloUTTracks = 0
    nFakeLongTracks = 0
    real=0.;fake=0. #real/fake minimums below threshold
    realbelow=0.;fakebelow=0. #how many times a real/fake is below threshold (hope for not many fakes) 
    chi_below_thr = 0 #tot number of chi2 below threshold
    missed_real=0.;missed_fake=0. #real/fake minimums above threshold (->missed)
    #decrease threshold to increase missed_fake (but then also missed_real increases -> compromise)

    threshold_chi2 = 0 #dummy
    
    nentries = ch1.GetEntries()
    index_perc = perc_list.index(percentage)

    #change chi threshold, sigmas and windows depending on smearing on/off
    if smearing == False:
        threshold_chi2 = thr_nosm_list[index_perc]
        #squared
        sigma2x=pow(sigmax_nosm,2)
        sigma2y=pow(sigmay_nosm,2)
        sigma2tx=pow(sigmatx_nosm,2)
        sigma2ty=pow(sigmaty_nosm,2)
        #2 sigma acceptance
        rangex=2*sigmax_nosm
        rangey=2*sigmay_nosm
    else:
        threshold_chi2 = thr_wsm_list[index_perc]
        sigma2x=pow(sigmax_wsm,2)
        sigma2y=pow(sigmay_wsm,2)
        sigma2tx=pow(sigmatx_wsm,2)
        sigma2ty=pow(sigmaty_wsm,2)
        #2 sigma acceptance
        rangex=2*sigmax_wsm
        rangey=2*sigmay_wsm

    #plots for strategy min chi2
    chi2R_true_list=[]
    chi2R_list=[]
    chi2F_list=[]

    pR_true_list=[]
    pR_list=[]
    pF_list=[]

    xzt0R_true_list=[]
    xzt0R_list=[]
    xzt0F_list=[]
    yzt0R_true_list=[]
    yzt0R_list=[]
    yzt0F_list=[]

    dxR_true_list=[]
    dxR_list=[]
    dxF_list=[]

    dyR_true_list=[]
    dyR_list=[]
    dyF_list=[]

    dtxR_true_list=[]
    dtxR_list=[]
    dtxF_list=[]

    dtyR_true_list=[]
    dtyR_list=[]
    dtyF_list=[]


    #plots for strategy cut below threshold
    percbelow_th_list=[]
    percF_th_list=[]

    print '\n\n\n\npercentage: ', percentage 
    print 'smearing: ', smearing 
    print 'threshold chi2: : ', threshold_chi2 

    ##### LOOP #####
    
    for event in ch1:

        ChisquareTry=[]
        Chi2Try_entryNumbers=[]
        pTry=[]
        xzt0Try=[]
        yzt0Try=[]
        dxTry=[]
        dyTry=[]
        dtxTry=[]
        dtyTry=[]

        #loop over all veloUT tracks
            #selecting some long tracks for event in ch1 and cut for !=0 else division/0
        if (event.HitUTZpos[0]-event.HitVeloZpos[0] != 0 and 0.<event.HitVeloZpos[0]<800. and event.HitUTZpos[0]>2000 and event.HitUTZpos[0]<3000 and event.p > 5000):
            nVeloUTTracks +=1
            isLongTrack=False

            #selecting some long tracks for event in ch1 and cut for !=0 else division/0
            if (event.HitZpos[0]>7000. and event.HitZpos[1]>7000 and event.HitZpos[1]-event.HitZpos[0] != 0):
                nLongTracks = nLongTracks+1
                isLongTrack=True
                 
                #use fist hit in velo and first hit in UT
                xv0=event.HitVeloXpos[0];  yv0 = event.HitVeloYpos[0]; zv0 = event.HitVeloZpos[0];
                xv1=event.HitUTXpos[0];  yv1 = event.HitUTYpos[0]; zv1 = event.HitUTZpos[0];
                xt0 = event.HitXpos[0]; yt0 = event.HitYpos[0]; zt0 = event.HitZpos[0];
                xt1 = event.HitXpos[1]; yt1 = event.HitYpos[1]; zt1 = event.HitZpos[1];
                vp = event.p

                chi2real_l = ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, rangex, rangey, sigma2x, sigma2y, sigma2tx, sigma2ty, check_if_inwindow=False,smearing=smearing)
                chi2real = chi2real_l[0]
                entryreal = event.GetReadEntry()
                eventreal = event.eventNumber
                p = event.p
                xzt0 = xt0 
                yzt0 = yt0
                dx = chi2real_l[1]
                dy = chi2real_l[2]
                dtx = chi2real_l[3]
                dty = chi2real_l[4]




            #loop over all events, cut for momentum and dz !=0 else division/0 (later cut in window)
            for event2 in ch2:
                #select T tracks in same event
                if (event2.eventNumber == eventreal and event2.HitZpos[0] > 7000 and event2.HitZpos[1] > 7000  and event2.p > 5000 and event2.HitZpos[1]-event2.HitZpos[0] != 0):

                    #overwrite the t track coordinates (velo coordinates and momentum (?) remain the same)
                    xt0 = event2.HitXpos[0]; yt0 = event2.HitYpos[0]; zt0 = event2.HitZpos[0];
                    xt1 = event2.HitXpos[1]; yt1 = event2.HitYpos[1]; zt1 = event2.HitZpos[1];

                    chi2try_l = ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, rangex, rangey, sigma2x, sigma2y, sigma2tx, sigma2ty, check_if_inwindow=True, smearing=smearing)
                    if chi2try_l[0] != -1: #cut events in search window (see def of chi2 function)
                        ChisquareTry.append(chi2try_l[0])
                        Chi2Try_entryNumbers.append(event2.GetReadEntry())
                        pTry.append(event2.p)
                        xzt0Try.append(xt0)
                        yzt0Try.append(yt0)
                        dxTry.append(chi2try_l[1])
                        dyTry.append(chi2try_l[2])
                        dtxTry.append(chi2try_l[3])
                        dtyTry.append(chi2try_l[4])

            if len(ChisquareTry) != 0: #(else it means that there was no t track in the windows)

                #STRATEGY MINSQUARE: find min chi2
                ChisquareSM=min(ChisquareTry)
                indexMin= ChisquareTry.index(ChisquareSM)

                if ChisquareSM < threshold_chi2:
                    #check if it real or fake match (compare number of entry)
                    if (Chi2Try_entryNumbers[indexMin]==entryreal and isLongTrack==True):
                        real=real+1.
                        #fill all the real plots
                        chi2R_true_list.append(chi2real)
                        pR_true_list.append(p)
                        xzt0R_true_list.append(xzt0)
                        yzt0R_true_list.append(yzt0)
                        dxR_true_list.append(dx)
                        dyR_true_list.append(dy)
                        dtxR_true_list.append(dtx)
                        dtyR_true_list.append(dty)

                    else:
                        fake=fake+1.
                        if(isLongTrack==False):
                            nFakeLongTracks+=1
                        #fill all the real plots
                        chi2R_list.append(chi2real)
                        pR_list.append(p)
                        xzt0R_list.append(xzt0)
                        yzt0R_list.append(yzt0)
                        dxR_list.append(dx)
                        dyR_list.append(dy)
                        dtxR_list.append(dtx)
                        dtyR_list.append(dty)
                        #fill all the fake plots
                        chi2F_list.append(ChisquareSM)
                        pF_list.append(pTry[indexMin])
                        xzt0F_list.append(xzt0Try[indexMin])
                        yzt0F_list.append(yzt0Try[indexMin])
                        dxF_list.append(dxTry[indexMin])
                        dyF_list.append(dyTry[indexMin])
                        dtxF_list.append(dtxTry[indexMin])
                        dtyF_list.append(dtyTry[indexMin])


                    #STRATEGY CUT BELOW THRESHOLD

                    chi_below_thr_perev = 0.
                    fakebelow_perev = 0.
                    realbelow_perev = 0.
                    count_perev = 0.

                    for chitry in ChisquareTry:
                        count_perev +=1
                        indexchitry = ChisquareTry.index(chitry)
                        if (chitry <= threshold_chi2):
                            #find how many tracks you find below threshold
                            chi_below_thr_perev+=1.
                            chi_below_thr+=1.
                            #find how many times the real one is amongst those below threshold
                            if (Chi2Try_entryNumbers[indexchitry]==entryreal and isLongTrack==True):
                                realbelow_perev+=1
                                realbelow+=1
                            #find how many wrong ones are below the threshold:
                            else:
                                fakebelow_perev+=1
                                fakebelow+=1

                    #plot some kind of (un)efficiency per event 
                    if chi_below_thr != 0:
                        percF_th_list.append(fakebelow_perev)
                    #plot % of t tracks below threshold 
                    if len(ChisquareTry) != 0:
                        percbelow_th_list.append(chi_below_thr_perev/len(ChisquareTry))


                else: #if chi2 is above threshold
                        if (Chi2Try_entryNumbers[indexMin]==entryreal and isLongTrack==True):
                            missed_real=missed_real+1.
                        else:
                            missed_fake=missed_fake+1.

            #cut at n good events (computational time issue)
            if (nLongTracks == 200 or entryreal == nentries):
                break
    
    #### PRINT RESULTS ####
    
    print 'number of velo tracks: ', nVeloUTTracks
    print 'number of long tracks: ', nLongTracks
    print'\nreal: ', real #min real below threshold
    print 'fake: ', fake #min fake below threshold
    print  'missed_real: ', missed_real #min real above threshold
    print  'missed fake: ', missed_fake #min fake above threshold
    print  'realbelow: ', realbelow #real is below threshold
    print 'fakebelow: ', fakebelow #fake is below threshold
    print 'non existing long tracks: ', nFakeLongTracks
    efficiency1 = real/nLongTracks
    efficiency2 = real/nVeloUTTracks
    ghostrate1 = fake/(fake+real)
    ghostrate2 = fake/nVeloUTTracks
    print '\n efficiency: real matches / long+downstream tracks', efficiency1
    print 'efficiency: real matches / VeloUT segments ', efficiency2
    print 'ghostrate: fake matches / all matches', ghostrate1
    print 'ghostrate: fake matches / VeloUT segments', ghostrate2
    deltat = time.time() - timestart
    print '\ntime: ', deltat/60






percentage:  97
smearing:  False
threshold chi2: :  32.18
number of velo tracks:  230
number of long tracks:  200

real:  178.0
fake:  30.0
missed_real:  0.0
missed fake:  0.0
realbelow:  178.0
fakebelow:  91.0
non existing long tracks:  29

 efficiency: real matches / long+downstream tracks 0.89
efficiency: real matches / VeloUT segments  0.773913043478
ghostrate: fake matches / all matches 0.144230769231
ghostrate: fake matches / VeloUT segments 0.130434782609

time:  4.39115043481


In [11]:
Nbins=60

#plots for strategy min chi2
chi2R_true=TH1F("chi2R_true","",Nbins,0,threshold_chi2)
chi2R=TH1F("chi2R","",Nbins,0,threshold_chi2)
chi2F=TH1F("chi2F","",Nbins,0,threshold_chi2)

pR_true=TH1F("pF_true","",Nbins,5000,50000)
pR=TH1F("pR","",Nbins,5000,50000)
pF=TH1F("pF","",Nbins,5000,50000)

xyzt0R_true=TH2F("xyzt0R_true","",Nbins,-3000,+3000,Nbins,-3000,+3000)
xyzt0R=TH2F("xyzt0R","",Nbins,-3000,+3000,Nbins,-3000,+3000)
xyzt0F=TH2F("xyzt0F","",Nbins,-3000,+3000,Nbins,-3000,+3000)

dxR_true=TH1F("dxR_true","",Nbins,-2*rangex,2*rangex)
dxR=TH1F("dxR","",Nbins,-2*rangex,2*rangex)
dxF=TH1F("dxF","",Nbins,-2*rangex,2*rangex)

dyR_true=TH1F("dyR_true","",Nbins,-2*rangey,2*rangey)
dyR=TH1F("dyR","",Nbins,-2*rangey,2*rangey)
dyF=TH1F("dyR","",Nbins,-2*rangey,2*rangey)

sigmatx = np.sqrt(sigma2tx)
dtxR_true=TH1F("dtxR_true","",Nbins,-2*sigmatx,2*sigmatx)
dtxR=TH1F("dtxR","",Nbins,-2*sigmatx,2*sigmatx)
dtxF=TH1F("dtxF","",Nbins,-2*sigmatx,2*sigmatx)

sigmaty = np.sqrt(sigma2ty)
dtyR_true=TH1F("dtyR_true","",Nbins,-2*sigmaty,2*sigmaty)
dtyR=TH1F("dtyR","",Nbins,-2*sigmaty,2*sigmaty)
dtyF=TH1F("dtyF","",Nbins,-2*sigmaty,2*sigmaty)


#plots for strategy cut below threshold
percbelow_th=TH1F("chi2R_th","",Nbins,0,1)
percF_th=TH1F("chi2F_th","",Nbins,0,1)

Warning in <TROOT::Append>: Replacing existing TH1: dyR (Potential memory leak).


In [13]:
#list of lists
listlist = [chi2R_true_list,chi2R_list,chi2F_list,pR_true_list,pR_list,pF_list,dxR_true_list,dxR_list,dxF_list,dyR_true_list,dyR_list,dyF_list,dtxR_true_list,dtxR_list,dtxF_list,dtyR_true_list,dtyR_list,dtyF_list,percbelow_th_list,percF_th_list]
#list of th1f histos
th1flist = [chi2R_true,chi2R,chi2F,pR_true,pR,pF,dxR_true,dxR,dxF,dyR_true,dyR,dyF,dtxR_true,dtxR,dtxF,dtyR_true,dtyR,dtyF,percbelow_th,percF_th]

#fill every th1f with the lists
for singlehisto in th1flist:
    index = th1flist.index(singlehisto)
    singlelist = listlist[index]
    for item in singlelist:
        singlehisto.Fill(item)



#list of lists
listlist2 = [xzt0R_true_list,yzt0R_true_list,xzt0R_list,yzt0R_list,xzt0F_list,yzt0F_list]
#list of th2f histos
th2flist = [xyzt0R_true,xyzt0R,xyzt0F]

#fill every histogram with the lists
for singlehisto in th2flist:
    index_x = (th2flist.index(singlehisto))*2
    index_y = (th2flist.index(singlehisto))*2+1
    singlelist_x = listlist2[index_x]
    singlelist_y = listlist2[index_y]
    for index in range(0,len(singlelist_x)):
        singlehisto.Fill(singlelist_x[index],singlelist_y[index])


In [19]:
#draw plots for minimum chi2 strategy

c4=TCanvas("c4","",2000,2000)

c4.Divide(3,3)

c4.cd(1)
chi2R.Draw()
chi2R.GetXaxis().SetTitle("#chi^{2}_{smallest}")
chi2R.GetYaxis().SetTitle("Tracks")
chi2R.GetYaxis().SetTitleOffset(0.8)
chi2R.SetLineColor(kBlue)
chi2R.SetMinimum(0.)
chi2F.Draw("SAME")
chi2F.SetLineColor(kRed)

legendi=TLegend(0.6,0.75,0.8,0.9)
legendi.AddEntry("chi2R","Real match","f")
legendi.AddEntry("chi2F","Fake match","f")
legendi.Draw()

c4.cd(2)
pR.Draw()
pR.GetXaxis().SetTitle("p")
pR.GetYaxis().SetTitle("Tracks")
pR.SetLineColor(kBlue)
pF.Draw("SAME")
pF.SetLineColor(kRed)

c4.cd(3)
xyzt0R.Draw()
xyzt0R.GetXaxis().SetTitle("x")
xyzt0R.GetYaxis().SetTitle("y")
xyzt0R.SetMarkerColor(kBlue)
xyzt0F.Draw("SAME")
xyzt0F.SetMarkerColor(kRed)


c4.cd(4)
dxR.Draw()
dxR.GetXaxis().SetTitle("dx")
dxR.GetYaxis().SetTitle("Tracks")
dxR.SetLineColor(kBlue)
dxF.Draw("SAME")
dxF.SetLineColor(kRed)


c4.cd(5)

dyR.Draw()
dyR.GetXaxis().SetTitle("dy")
dyR.GetYaxis().SetTitle("Tracks")
dyR.SetLineColor(kBlue)
dyF.Draw("SAME")
dyF.SetLineColor(kRed)

c4.cd(6)
dtxR.Draw()
dtxR.GetXaxis().SetTitle("dtx")
dtxR.GetYaxis().SetTitle("Tracks")
dtxR.SetLineColor(kBlue)
dtxF.Draw("SAME")
dtxF.SetLineColor(kRed)

c4.cd(7)
dtyR.Draw()
dtyR.GetXaxis().SetTitle("dty")
dtyR.GetYaxis().SetTitle("Tracks")
dtyR.SetLineColor(kBlue)
dtyF.Draw("SAME")
dtyF.SetLineColor(kRed)

c4.SaveAs("distinguish_range_fr_fakeminVeloUT.png")



Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file distinguish_range_fr_fakeminUT.png has been created


In [20]:

#draw plots for minimum chi2 strategy

c3=TCanvas("c3","",2000,2000)

c3.Divide(3,3)

c3.cd(1)
chi2R_true.Draw()
chi2R_true.GetXaxis().SetTitle("#chi^{2}_{smallest}")
chi2R_true.GetYaxis().SetTitle("Tracks")
chi2R_true.GetYaxis().SetTitleOffset(0.8)
chi2R_true.SetLineColor(kBlue)
chi2R_true.SetMinimum(0.)

c3.cd(2)
pR_true.Draw()
pR_true.GetXaxis().SetTitle("p")
pR_true.GetYaxis().SetTitle("Tracks")
pR_true.SetLineColor(kBlue)

c3.cd(3)
xyzt0R_true.Draw()
xyzt0R_true.GetXaxis().SetTitle("x")
xyzt0R_true.GetYaxis().SetTitle("y")
xyzt0R.SetMarkerColor(kBlue)


c3.cd(4)
dxR_true.Draw()
dxR_true.GetXaxis().SetTitle("dx")
dxR_true.GetYaxis().SetTitle("Tracks")
dxR_true.SetLineColor(kBlue)

c3.cd(5)
dyR_true.Draw()
dyR_true.GetXaxis().SetTitle("dy")
dyR_true.GetYaxis().SetTitle("Tracks")
dyR_true.SetLineColor(kBlue)

c3.cd(6)
dtxR_true.Draw()
dtxR_true.GetXaxis().SetTitle("dtx")
dtxR_true.GetYaxis().SetTitle("Tracks")
dtxR_true.SetLineColor(kBlue)

c3.cd(7)
dtyR_true.Draw()
dtyR_true.GetXaxis().SetTitle("dty")
dtyR_true.GetYaxis().SetTitle("Tracks")
dtyR_true.SetLineColor(kBlue)

c3.SaveAs("distinguish_range_fr_trueminVeloUT.png")


Info in <TCanvas::Print>: png file distinguish_range_fr_trueminUT.png has been created


In [15]:

#draw plots for cut below threshold strategy

c5=TCanvas("c5","",1300,900)
c5.Divide(2,1)
c5.cd(1)
percF_th.Draw()
percF_th.GetXaxis().SetTitle("fakebelow")
percF_th.GetYaxis().SetTitle("Tracks")
percF_th.GetYaxis().SetTitleOffset(0.8)
percF_th.SetLineColor(kBlue)
percF_th.SetMinimum(0.)

c5.cd(2)
percbelow_th.Draw()
percbelow_th.GetXaxis().SetTitle("totbelow/chi2try")
percbelow_th.GetYaxis().SetTitle("Tracks")
percbelow_th.GetYaxis().SetTitleOffset(0.8)
percbelow_th.SetLineColor(kRed)
percbelow_th.SetMinimum(0.)
c5.SaveAs("distinguish_range_belowthrVeloUT.png")

Info in <TCanvas::Print>: png file distinguish_range_belowthrUT.png has been created


In [16]:
def cloneandscale(th1f,scale=1):
    scaledhisto = (th1f.Clone(""))
    for iteration in range(0,scale-1):
        scaledhisto.Add(th1f)
    return scaledhisto


In [21]:
#draw plots for minimum chi2 strategy

scalingfactor = 5

c7=TCanvas("c7","",2000,2000)

c7.Divide(3,3)

c7.cd(1)
chi2R_true.Draw()
chi2R_true.GetXaxis().SetTitle("#chi^{2}_{smallest}")
chi2R_true.GetYaxis().SetTitle("Tracks")
chi2R_true.GetYaxis().SetTitleOffset(0.8)
chi2R_true.SetLineColor(kBlue)
chi2R_true.SetMinimum(0.)
chi2Rscaled = cloneandscale(chi2R,scalingfactor)
chi2Rscaled.Draw("SAME")
chi2Rscaled.SetLineColor(kRed)

legendi=TLegend(0.6,0.75,0.8,0.9)
legendi.AddEntry("chi2R","Real match","f")
legendi.AddEntry("chi2F","Fake match","f")
legendi.Draw()

c7.cd(2)
pR_true.Draw()
pR_true.GetXaxis().SetTitle("p")
pR_true.GetYaxis().SetTitle("Tracks")
pR_true.SetLineColor(kBlue)
pRscaled = cloneandscale(pR,scalingfactor)
pRscaled.Draw("SAME")
pRscaled.SetLineColor(kRed)


c7.cd(3)
xyzt0R_true.Draw()
xyzt0R_true.GetXaxis().SetTitle("x")
xyzt0R_true.GetYaxis().SetTitle("y")
xyzt0R_true.SetMarkerColor(kBlue)
xyzt0Rscaled = cloneandscale(xyzt0R,1)
xyzt0Rscaled.Draw("SAME")
xyzt0Rscaled.SetMarkerColor(kRed)


c7.cd(4)
dxR_true.Draw()
dxR_true.GetXaxis().SetTitle("dx")
dxR_true.GetYaxis().SetTitle("Tracks")
dxR_true.SetLineColor(kBlue)
dxRscaled = cloneandscale(dxR,scalingfactor)
dxRscaled.Draw("SAME")
dxRscaled.SetLineColor(kRed)


c7.cd(5)
dyR_true.Draw()
dyR_true.GetXaxis().SetTitle("dy")
dyR_true.GetYaxis().SetTitle("Tracks")
dyR_true.SetLineColor(kBlue)
dyRscaled = cloneandscale(dyR,scalingfactor)
dyRscaled.Draw("SAME")
dyRscaled.SetLineColor(kRed)

c7.cd(6)
dtxR_true.Draw()
dtxR_true.GetXaxis().SetTitle("dx")
dtxR_true.GetYaxis().SetTitle("Tracks")
dtxR_true.SetLineColor(kBlue)
dtxRscaled = cloneandscale(dtxR,scalingfactor)
dtxRscaled.Draw("SAME")
dtxRscaled.SetLineColor(kRed)

c7.cd(7)
dtyR_true.Draw()
dtyR_true.GetXaxis().SetTitle("dy")
dtyR_true.GetYaxis().SetTitle("Tracks")
dtyR_true.SetLineColor(kBlue)
dtyRscaled = cloneandscale(dtyR,scalingfactor)
dtyRscaled.Draw("SAME")
dtyRscaled.SetLineColor(kRed)

c7.SaveAs("distinguish_range_fr_trueandfakeminVeloUT.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c7
Info in <TCanvas::Print>: png file distinguish_range_fr_trueandfakeminUT.png has been created
